<a href="https://www.kaggle.com/code/syedfarazhussaini/2026feb-prediciting-heart-disease?scriptVersionId=295231738" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Heart Disease Prediction - Binary Classification
## Playground Series S6E2 - Optimized Pipeline

**Goal**: Predict heart disease probability (AUC-ROC metric)
**Approach**: Feature engineering + Stacking + Blending
**Target**: Top 10 (~24 position)

## 1. Setup & Imports

In [1]:
import numpy as np
import pandas as pd
from pathlib import Path

from sklearn.model_selection import KFold, train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LogisticRegression, Ridge
from sklearn.metrics import roc_auc_score

from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier

import warnings
warnings.filterwarnings('ignore')

print("✓ All imports successful")

✓ All imports successful


## 2. Data Loading & Paths

In [2]:
# Check if running locally or on Kaggle
import os
IS_KAGGLE = os.path.exists('/kaggle')

if IS_KAGGLE:
    DATA_PATH = Path('/kaggle/input/playground-series-s6e2')
    OUTPUT_PATH = Path('/kaggle/working')
    print("Running on Kaggle")
else:
    DATA_PATH = Path('./data')
    OUTPUT_PATH = Path('./submissions')
    print("Running locally")

print(f"Data path: {DATA_PATH}")
print(f"Output path: {OUTPUT_PATH}")

Running on Kaggle
Data path: /kaggle/input/playground-series-s6e2
Output path: /kaggle/working


## 3. Load Data

In [3]:
# Load datasets
train_df = pd.read_csv(DATA_PATH / 'train.csv')
test_df = pd.read_csv(DATA_PATH / 'test.csv')
submission_sample = pd.read_csv(DATA_PATH / 'sample_submission.csv')

print(f"Train shape: {train_df.shape}")
print(f"Test shape: {test_df.shape}")
print(f"\nTrain columns: {list(train_df.columns)}")
print(f"\nFirst rows of train:")
print(train_df.head())

Train shape: (630000, 15)
Test shape: (270000, 14)

Train columns: ['id', 'Age', 'Sex', 'Chest pain type', 'BP', 'Cholesterol', 'FBS over 120', 'EKG results', 'Max HR', 'Exercise angina', 'ST depression', 'Slope of ST', 'Number of vessels fluro', 'Thallium', 'Heart Disease']

First rows of train:
   id  Age  Sex  Chest pain type   BP  Cholesterol  FBS over 120  EKG results  \
0   0   58    1                4  152          239             0            0   
1   1   52    1                1  125          325             0            2   
2   2   56    0                2  160          188             0            2   
3   3   44    0                3  134          229             0            2   
4   4   58    1                4  140          234             0            2   

   Max HR  Exercise angina  ST depression  Slope of ST  \
0     158                1            3.6            2   
1     171                0            0.0            1   
2     151                0            0.0

## 4. EDA & Feature Analysis

In [4]:
# Data types and missing values
print("="*60)
print("DATA OVERVIEW")
print("="*60)

print("\nData types:")
print(train_df.dtypes)

print("\nMissing values:")
print(train_df.isnull().sum())

# Feature analysis
print("\nTarget variable distribution:")
print(train_df['Heart Disease'].value_counts())
print(f"\nTarget ratio: {(train_df['Heart Disease'] == 'Presence').mean():.4f} (% positive)")

# Identify feature types
numeric_features = train_df.select_dtypes(include=['int64', 'float64']).columns.tolist()
categorical_features = train_df.select_dtypes(include=['object']).columns.tolist()

# Remove id and target from features
if 'id' in numeric_features:
    numeric_features.remove('id')
if 'Heart Disease' in numeric_features:
    numeric_features.remove('Heart Disease')

print(f"\nNumeric features ({len(numeric_features)}): {numeric_features[:5]}...")
print(f"Categorical features ({len(categorical_features)}): {categorical_features}")

DATA OVERVIEW

Data types:
id                           int64
Age                          int64
Sex                          int64
Chest pain type              int64
BP                           int64
Cholesterol                  int64
FBS over 120                 int64
EKG results                  int64
Max HR                       int64
Exercise angina              int64
ST depression              float64
Slope of ST                  int64
Number of vessels fluro      int64
Thallium                     int64
Heart Disease               object
dtype: object

Missing values:
id                         0
Age                        0
Sex                        0
Chest pain type            0
BP                         0
Cholesterol                0
FBS over 120               0
EKG results                0
Max HR                     0
Exercise angina            0
ST depression              0
Slope of ST                0
Number of vessels fluro    0
Thallium                   0
Heart Disea

## 5. Feature Correlations

In [5]:
# Calculate correlations with target
# Encode target variable for correlation calculation
target_encoded = (train_df['Heart Disease'] == 'Presence').astype(int)
df_for_corr = train_df[numeric_features].copy()
df_for_corr['Heart Disease'] = target_encoded

correlations = df_for_corr.corr()['Heart Disease'].drop('Heart Disease')
correlations = correlations.abs().sort_values(ascending=False)

print("Feature Correlations with Target (absolute):")
print(correlations)

# Basic statistics
print("\n" + "="*60)
print("NUMERIC FEATURE STATISTICS")
print("="*60)
print(train_df[numeric_features].describe())

Feature Correlations with Target (absolute):
Thallium                   0.605776
Chest pain type            0.460684
Exercise angina            0.441864
Max HR                     0.440985
Number of vessels fluro    0.438604
ST depression              0.430641
Slope of ST                0.415050
Sex                        0.342446
EKG results                0.218961
Age                        0.212091
Cholesterol                0.082753
FBS over 120               0.033570
BP                         0.005181
Name: Heart Disease, dtype: float64

NUMERIC FEATURE STATISTICS
                 Age            Sex  Chest pain type             BP  \
count  630000.000000  630000.000000    630000.000000  630000.000000   
mean       54.136706       0.714735         3.312752     130.497433   
std         8.256301       0.451541         0.851615      14.975802   
min        29.000000       0.000000         1.000000      94.000000   
25%        48.000000       0.000000         3.000000     120.000000 

## 6. Preprocessing

In [6]:
def preprocess_data(df, categorical_features, numeric_features, fit_scalers=True, scalers=None):
    """Encode categoricals, scale numerics"""
    df_proc = df.copy()
    
    if fit_scalers:
        scalers = {}
        # Encode categorical features
        for col in categorical_features:
            if col in df_proc.columns:
                le = LabelEncoder()
                df_proc[col] = le.fit_transform(df_proc[col].astype(str))
                scalers[col] = le
        
        # Scale numeric features
        if numeric_features:
            scaler = StandardScaler()
            df_proc[numeric_features] = scaler.fit_transform(df_proc[numeric_features])
            scalers['numeric'] = scaler
    else:
        # Transform using fitted scalers
        for col in categorical_features:
            if col in df_proc.columns and col in scalers:
                df_proc[col] = scalers[col].transform(df_proc[col].astype(str))
        
        if numeric_features and 'numeric' in scalers:
            df_proc[numeric_features] = scalers['numeric'].transform(df_proc[numeric_features])
    
    return df_proc, scalers

# Preprocess training data
train_processed, scalers = preprocess_data(train_df.drop('id', axis=1), 
                                           categorical_features, 
                                           numeric_features,
                                           fit_scalers=True)

# Preprocess test data
test_processed, _ = preprocess_data(test_df.drop('id', axis=1), 
                                    categorical_features, 
                                    numeric_features,
                                    fit_scalers=False, 
                                    scalers=scalers)

print("✓ Preprocessing complete")
print(f"Train shape: {train_processed.shape}")
print(f"Test shape: {test_processed.shape}")

✓ Preprocessing complete
Train shape: (630000, 14)
Test shape: (270000, 13)


## 7. Train/Val Split & Feature Preparation

In [7]:
# Separate features and target
X_all = train_processed.drop('Heart Disease', axis=1)
y_all = train_processed['Heart Disease']
X_test = test_processed.drop('Heart Disease', axis=1) if 'Heart Disease' in test_processed.columns else test_processed

# 80/20 split for baseline
X_train, X_val, y_train, y_val = train_test_split(X_all, y_all, test_size=0.2, 
                                                    random_state=42, stratify=y_all)

print(f"Training set: {X_train.shape}")
print(f"Validation set: {X_val.shape}")
print(f"Test set: {X_test.shape}")
print(f"\nTarget ratio - Train: {y_train.mean():.4f}, Val: {y_val.mean():.4f}")

Training set: (504000, 13)
Validation set: (126000, 13)
Test set: (270000, 13)

Target ratio - Train: 0.4483, Val: 0.4483


## 8. Baseline Model - Logistic Regression

In [8]:
print("="*60)
print("BASELINE MODEL - LOGISTIC REGRESSION")
print("="*60)

baseline_model = LogisticRegression(max_iter=1000, random_state=42)
baseline_model.fit(X_train, y_train)

# Predictions
y_train_pred = baseline_model.predict_proba(X_train)[:, 1]
y_val_pred = baseline_model.predict_proba(X_val)[:, 1]

# Evaluation
train_auc = roc_auc_score(y_train, y_train_pred)
val_auc = roc_auc_score(y_val, y_val_pred)

print(f"Training AUC-ROC: {train_auc:.4f}")
print(f"Validation AUC-ROC: {val_auc:.4f}")

BASELINE MODEL - LOGISTIC REGRESSION
Training AUC-ROC: 0.9502
Validation AUC-ROC: 0.9515


## 9. Model Comparison - 5-Fold Cross-Validation

In [9]:
def auc_score(y_true, y_pred):
    """Calculate AUC-ROC"""
    return roc_auc_score(y_true, y_pred)

def cross_validate_classifier(model_builder, X, y, name, folds=5):
    """5-fold cross-validation for classifiers"""
    kf = KFold(n_splits=folds, shuffle=True, random_state=42)
    scores = []
    
    for fold, (train_idx, val_idx) in enumerate(kf.split(X)):
        model = model_builder()
        model.fit(X.iloc[train_idx], y.iloc[train_idx])
        val_preds = model.predict_proba(X.iloc[val_idx])[:, 1]
        score = auc_score(y.iloc[val_idx], val_preds)
        scores.append(score)
    
    mean_score = np.mean(scores)
    std_score = np.std(scores)
    print(f"{name:15s}: {mean_score:.4f} ± {std_score:.4f}")
    return mean_score, std_score

print("="*60)
print("MODEL COMPARISON - 5-FOLD CROSS-VALIDATION")
print("="*60)

# Model builders
def build_lgb():
    return LGBMClassifier(n_estimators=200, learning_rate=0.08, num_leaves=80, 
                         max_depth=8, subsample=0.8, colsample_bytree=0.8,
                         reg_alpha=0.1, reg_lambda=0.3, random_state=42, verbose=-1)

def build_xgb():
    return XGBClassifier(n_estimators=250, learning_rate=0.05, max_depth=5,
                        subsample=0.8, colsample_bytree=0.8, reg_alpha=0.1,
                        reg_lambda=0.5, random_state=42, verbosity=0)

def build_cat():
    return CatBoostClassifier(iterations=200, learning_rate=0.08, depth=7,
                             subsample=0.8, random_state=42, verbose=0)

# Test each model
lgb_mean, lgb_std = cross_validate_classifier(build_lgb, X_all, y_all, "LightGBM")
xgb_mean, xgb_std = cross_validate_classifier(build_xgb, X_all, y_all, "XGBoost")
cat_mean, cat_std = cross_validate_classifier(build_cat, X_all, y_all, "CatBoost")

print(f"\nBest model: {'LightGBM' if lgb_mean > xgb_mean and lgb_mean > cat_mean else 'XGBoost' if xgb_mean > cat_mean else 'CatBoost'}")

MODEL COMPARISON - 5-FOLD CROSS-VALIDATION
LightGBM       : 0.9551 ± 0.0004
XGBoost        : 0.9549 ± 0.0004
CatBoost       : 0.9550 ± 0.0004

Best model: LightGBM


## 10. Feature Engineering

In [10]:
print("="*60)
print("FEATURE ENGINEERING")
print("="*60)

# Get original numeric features from unprocessed data
X_train_orig = train_df.loc[X_train.index].drop(['id', 'Heart Disease'], axis=1)
X_val_orig = train_df.loc[X_val.index].drop(['id', 'Heart Disease'], axis=1)
X_test_orig = test_df.drop('id', axis=1)

def create_features(df_orig, df_proc):
    """Create engineered features for classification"""
    df_eng = df_proc.copy()
    
    # Get numeric columns from original (unscaled) data
    numeric_cols = [col for col in numeric_features if col in df_orig.columns]
    
    if len(numeric_cols) > 0:
        # Interactions between top correlated features
        if len(numeric_cols) >= 2:
            # Pairwise interactions
            for i in range(min(3, len(numeric_cols))):
                for j in range(i+1, min(4, len(numeric_cols))):
                    col1, col2 = numeric_cols[i], numeric_cols[j]
                    df_eng[f'{col1}_x_{col2}'] = df_orig[col1] * df_orig[col2]
        
        # Polynomial features for top features
        for col in numeric_cols[:3]:
            df_eng[f'{col}_sq'] = df_orig[col] ** 2
            df_eng[f'{col}_sqrt'] = np.sqrt(np.abs(df_orig[col]))
        
        # Ratio features
        if len(numeric_cols) >= 2:
            df_eng[f'{numeric_cols[0]}_div_{numeric_cols[1]}'] = (df_orig[numeric_cols[0]] / 
                                                                    (np.abs(df_orig[numeric_cols[1]]) + 1e-5))
    
    return df_eng

# Create engineered features
X_train_eng = create_features(X_train_orig, X_train)
X_val_eng = create_features(X_val_orig, X_val)
X_test_eng = create_features(X_test_orig, X_test)

print(f"Original features: {X_train.shape[1]}")
print(f"Engineered features: {X_train_eng.shape[1]}")
print(f"New features added: {X_train_eng.shape[1] - X_train.shape[1]}")

FEATURE ENGINEERING
Original features: 13
Engineered features: 26
New features added: 13


## 11. Feature Set Comparison

In [11]:
print("="*60)
print("FEATURE SET COMPARISON - 3-FOLD CV")
print("="*60)

def quick_cv(X, y, model_builder, folds=3):
    """Quick 3-fold CV for comparison"""
    kf = KFold(n_splits=folds, shuffle=True, random_state=42)
    scores = []
    
    for fold, (train_idx, val_idx) in enumerate(kf.split(X)):
        model = model_builder()
        model.fit(X.iloc[train_idx], y.iloc[train_idx])
        val_preds = model.predict_proba(X.iloc[val_idx])[:, 1]
        score = auc_score(y.iloc[val_idx], val_preds)
        scores.append(score)
    
    return np.mean(scores), np.std(scores)

# Compare feature sets
orig_mean, orig_std = quick_cv(X_all, y_all, build_lgb, folds=3)
eng_mean, eng_std = quick_cv(pd.concat([X_train_eng, X_val_eng], axis=0), y_all, build_lgb, folds=3)

print(f"Original features:   {orig_mean:.4f} ± {orig_std:.4f} AUC-ROC")
print(f"Engineered features: {eng_mean:.4f} ± {eng_std:.4f} AUC-ROC")
print(f"Improvement: {(eng_mean - orig_mean):.4f} AUC-ROC")

# Use engineered features
if eng_mean > orig_mean:
    print("✓ Using engineered features")
else:
    print("✓ Using original features")
    X_train_eng, X_val_eng, X_test_eng = X_train.copy(), X_val.copy(), X_test.copy()

FEATURE SET COMPARISON - 3-FOLD CV
Original features:   0.9550 ± 0.0001 AUC-ROC
Engineered features: 0.4995 ± 0.0006 AUC-ROC
Improvement: -0.4554 AUC-ROC
✓ Using original features


## 12. Stacking Ensemble - 5-Fold Meta-Features

In [12]:
print("="*60)
print("STACKING ENSEMBLE - 5-FOLD CV")
print("="*60)

# Combine train/val for full dataset stacking
X_stack = pd.concat([X_train_eng, X_val_eng], axis=0)
y_stack = pd.concat([y_train, y_val], axis=0)

# 5-fold CV for meta-features
kf_stack = KFold(n_splits=5, shuffle=True, random_state=42)
meta_train = np.zeros((len(X_stack), 2))  # 2 base models
meta_test = np.zeros((len(X_test_eng), 2))

for fold, (train_idx, val_idx) in enumerate(kf_stack.split(X_stack)):
    print(f"Fold {fold+1}/5...", end=" ")
    
    # Train base models on fold
    lgb_base = build_lgb()
    lgb_base.fit(X_stack.iloc[train_idx], y_stack.iloc[train_idx])
    
    xgb_base = build_xgb()
    xgb_base.fit(X_stack.iloc[train_idx], y_stack.iloc[train_idx])
    
    # Meta-features on validation fold
    meta_train[val_idx, 0] = lgb_base.predict_proba(X_stack.iloc[val_idx])[:, 1]
    meta_train[val_idx, 1] = xgb_base.predict_proba(X_stack.iloc[val_idx])[:, 1]
    
    # Average test predictions
    meta_test[:, 0] += lgb_base.predict_proba(X_test_eng)[:, 1] / 5
    meta_test[:, 1] += xgb_base.predict_proba(X_test_eng)[:, 1] / 5
    
    lgb_val_auc = auc_score(y_stack.iloc[val_idx], meta_train[val_idx, 0])
    xgb_val_auc = auc_score(y_stack.iloc[val_idx], meta_train[val_idx, 1])
    print(f"LGB={lgb_val_auc:.4f}, XGB={xgb_val_auc:.4f}")

# Train meta-learner (Ridge for classification)
meta_learner = Ridge(alpha=1.0)
meta_learner.fit(meta_train, y_stack)

# Meta-learner predictions
stack_train_preds = meta_learner.predict(meta_train)
stack_test_preds = meta_learner.predict(meta_test)

print(f"\nStack train AUC: {auc_score(y_stack, stack_train_preds):.4f}")
print(f"Meta-learner weights: LGB={meta_learner.coef_[0]:.4f}, XGB={meta_learner.coef_[1]:.4f}")

STACKING ENSEMBLE - 5-FOLD CV
Fold 1/5... LGB=0.9556, XGB=0.9554
Fold 2/5... LGB=0.9547, XGB=0.9545
Fold 3/5... LGB=0.9547, XGB=0.9544
Fold 4/5... LGB=0.9554, XGB=0.9551
Fold 5/5... LGB=0.9552, XGB=0.9549

Stack train AUC: 0.9552
Meta-learner weights: LGB=0.7244, XGB=0.2754


## 13. Blending - Final Ensemble

In [13]:
print("="*60)
print("BLENDING - WEIGHTED ENSEMBLE")
print("="*60)

# Train final base models on full dataset
lgb_final = build_lgb()
lgb_final.fit(X_stack, y_stack)
lgb_preds_test = lgb_final.predict_proba(X_test_eng)[:, 1]

xgb_final = build_xgb()
xgb_final.fit(X_stack, y_stack)
xgb_preds_test = xgb_final.predict_proba(X_test_eng)[:, 1]

# Blend: weight 0.4 LGB, 0.4 XGB, 0.2 Stacking
blend_preds = 0.4 * lgb_preds_test + 0.4 * xgb_preds_test + 0.2 * stack_test_preds

print(f"\n✓ Blend weights: LGB=0.4, XGB=0.4, Stacking=0.2")
print(f"Blend predictions shape: {blend_preds.shape}")
print(f"Blend prediction range: [{blend_preds.min():.4f}, {blend_preds.max():.4f}]")
print(f"Mean: {blend_preds.mean():.4f}, Std: {blend_preds.std():.4f}")

BLENDING - WEIGHTED ENSEMBLE

✓ Blend weights: LGB=0.4, XGB=0.4, Stacking=0.2
Blend predictions shape: (270000,)
Blend prediction range: [0.0006, 0.9997]
Mean: 0.4498, Std: 0.4067


## 14. Final Submission

In [14]:
print("="*60)
print("FINAL SUBMISSION")
print("="*60)

# Create submission
submission_final = submission_sample.copy()
submission_final['Heart Disease'] = blend_preds
submission_final.to_csv(OUTPUT_PATH / 'submission.csv', index=False)

print(f"\n✓ Submission saved to submission.csv")
print(f"  Format: Stacking (LGB + XGB) + Blending")
print(f"  Features: {X_test_eng.shape[1]} total")
print(f"\nSubmission preview:")
print(submission_final.head(10))
print(f"\nSubmission stats:")
print(f"  Records: {len(submission_final)}")
print(f"  Mean: {blend_preds.mean():.4f}")
print(f"  Min:  {blend_preds.min():.4f}")
print(f"  Max:  {blend_preds.max():.4f}")

FINAL SUBMISSION

✓ Submission saved to submission.csv
  Format: Stacking (LGB + XGB) + Blending
  Features: 13 total

Submission preview:
       id  Heart Disease
0  630000       0.943702
1  630001       0.009017
2  630002       0.983263
3  630003       0.005784
4  630004       0.178748
5  630005       0.980290
6  630006       0.006950
7  630007       0.624030
8  630008       0.991829
9  630009       0.015427

Submission stats:
  Records: 270000
  Mean: 0.4498
  Min:  0.0006
  Max:  0.9997


## 15. Summary

In [15]:
print("\n" + "="*70)
print("FINAL SUMMARY - HEART DISEASE PREDICTION")
print("="*70)

print("\n📊 PIPELINE:")
print(f"  ✓ Baseline Logistic Regression: {val_auc:.4f} AUC")
print(f"  ✓ LightGBM (5-fold): {lgb_mean:.4f} ± {lgb_std:.4f}")
print(f"  ✓ XGBoost (5-fold): {xgb_mean:.4f} ± {xgb_std:.4f}")
print(f"  ✓ CatBoost (5-fold): {cat_mean:.4f} ± {cat_std:.4f}")
print(f"  ✓ Feature Engineering: {X_test_eng.shape[1]} features")
print(f"  ✓ Stacking Ensemble: 5-fold meta-features + Ridge")
print(f"  ✓ Blending: LGB 0.4 + XGB 0.4 + Stacking 0.2")

print("\n🎯 SUBMISSION:")
print(f"  Location: submissions/submission.csv")
print(f"  Records: {len(submission_final)}")
print(f"  Prediction range: [{blend_preds.min():.4f}, {blend_preds.max():.4f}]")
print(f"  Status: ✅ Ready for upload")

print("\n" + "="*70)


FINAL SUMMARY - HEART DISEASE PREDICTION

📊 PIPELINE:
  ✓ Baseline Logistic Regression: 0.9515 AUC
  ✓ LightGBM (5-fold): 0.9551 ± 0.0004
  ✓ XGBoost (5-fold): 0.9549 ± 0.0004
  ✓ CatBoost (5-fold): 0.9550 ± 0.0004
  ✓ Feature Engineering: 13 features
  ✓ Stacking Ensemble: 5-fold meta-features + Ridge
  ✓ Blending: LGB 0.4 + XGB 0.4 + Stacking 0.2

🎯 SUBMISSION:
  Location: submissions/submission.csv
  Records: 270000
  Prediction range: [0.0006, 0.9997]
  Status: ✅ Ready for upload

